In [3]:
import yfinance as yf
import pandas as pd
import numpy as np 
import statsmodels.api as sm

Tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
Weights = [0.25, 0.25, 0.25, 0.25]
Start = '2015-01-01' 
Years = 5
Sims = 5000
Trading_Days = 252
Block = 10 
Seed = 42 
Days = int(Years * Trading_Days)
rf_annual = 0.04
rf_daily = (1 + rf_annual) ** (1/252) - 1

prices = yf.download(Tickers,start=Start,auto_adjust=True,progress=False)["Close"].dropna()
returns = prices.pct_change().dropna()
pricesSPY = yf.download('SPY', start=Start,auto_adjust=True,progress=False)["Close"].dropna()
returnsSPY = pricesSPY.pct_change().dropna()

def block_bootstrap_paths(returns_df, weights, days, sims, block, seed):
    rng = np.random.default_rng(seed) 
    ret_np = returns_df.values 
    T, n = ret_np.shape #T=Number of past days n=number of tickers
    w = np.array(weights, dtype=float) #Normalize portfolio weights summing to 1
    w /= w.sum()
    n_blocks = int(np.ceil(days / block))
    paths = np.zeros((days, sims)) #Matrix to store simulated path
    sharpe_ratios = np.zeros(sims) 
    off = np.arange(block) #Create an array for indexing wihthin each block

    for s in range(sims):
        starts = rng.integers(0, T, size=n_blocks) #Choose random positions for each block
        idx = (starts[:, None] + off[None, :]) % T #Block offsets to start pos for full index range
        sampled = ret_np[idx.reshape(-1), :][:days] #Pull historical returns and trims
        port_daily = sampled @ w 
        paths[:, s] = np.cumprod(1 + port_daily) #Converts returns to growth 
        paths[:, s] /= paths[0, s]

        excess_returns = port_daily - rf_daily #Computes Sharpe ratio for the path
        mean_excess = excess_returns.mean()
        std_excess = excess_returns.std()
        sharpe_ratios[s] = (mean_excess / std_excess) * np.sqrt(252)

    return paths, sharpe_ratios

paths, sharpe_ratios = block_bootstrap_paths(returns, Weights, Days, Sims, Block, Seed)
pathsSPY, SPYsharpe_ratios = block_bootstrap_paths(returnsSPY, [1.0], Days, Sims, Block, Seed)
q = [25,50,75]
bands = np.percentile(paths, q, 1) #Computes percentile bands 
bandsSPY = np.percentile(pathsSPY, q, 1) 
p25, p50, p75 = np.percentile(paths[-1], q) #Computes percentiles of growth for ending
bandsSPY = np.percentile(pathsSPY, q, 1)
pS25, pS50, pS75 = np.percentile(pathsSPY[-1], q)

ff = pd.read_csv("https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip", compression="zip", skiprows=3)
date_col = ff.columns[0]
ff = ff.rename(columns={date_col: "Date"})
ff = ff[ff["Date"].astype(str).str.match(r"^\d{8}$")] # Keep only rows that look like (YYYYMMDD)
ff["Date"] = pd.to_datetime(ff["Date"], format="%Y%m%d") # Convert date to datetime
ff[["Mkt-RF", "SMB", "HML", "RF"]] = ff[["Mkt-RF", "SMB", "HML", "RF"]].astype(float) / 100 # Convert % to decimals
ff = ff.set_index("Date").sort_index() # Use Date as index

w = np.array(Weights, dtype=float) #Normalize portfolio weights summing to 1
w /= w.sum()
Rp = returns.dot(w)
Rp.name = "Rp"
data = pd.concat([Rp, ff[['Mkt-RF','SMB','HML','RF']]], axis=1, join="inner").dropna()
rows = []
for month, grp in data.groupby(data.index.to_period('M')):
    y = grp['Rp'] - grp['RF']                   
    X = sm.add_constant(grp[['Mkt-RF','SMB','HML']])
    m = sm.OLS(y, X).fit()
    rows.append({
        'Date': month.to_timestamp(),
        'Beta_Mkt': m.params['Mkt-RF'],
        'Beta_SMB': m.params['SMB'],
        'Beta_HML': m.params['HML']
    })

pd.DataFrame({
    "day": np.arange(Days),
    "p25": bands[0],
    "p50": bands[1],
    "p75": bands[2]
}).to_csv("bands.csv", index=False) 

pd.DataFrame({
    "day": np.arange(Days),
    "pS25": bandsSPY[0],
    "pS50": bandsSPY[1],
    "pS75": bandsSPY[2]
}).to_csv("bandsSPY.csv", index=False)

pd.DataFrame({
    "Median": [np.median(sharpe_ratios)],
    "p25": [np.percentile(sharpe_ratios, 25)],
    "p75": [np.percentile(sharpe_ratios, 75)],
    "MedianS": [np.median(SPYsharpe_ratios)],
    "pS25": [np.percentile(SPYsharpe_ratios, 25)],
    "pS75": [np.percentile(SPYsharpe_ratios, 75)]
}).to_csv("Sharpe.csv", index=False)

pd.DataFrame(rows).to_csv("3Factor.csv", index=False)